In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import scanpy as sc
import squidpy as sq
import pandas as pd

In [2]:
input_dir = '../../results/05_snakemake_clustering/output/svg'
out_dir = '../../results/05_snakemake_clustering/02_get_genes'

In [3]:
os.makedirs(out_dir, exist_ok=True)

In [4]:
method_list = ['nnSVG', 'Spanve',  'SPARK-X', 'SPARK', 'SpatialDE2',
               'SpatialDE', 'SOMDE', 'MoranI', 'scGCO', 'SpaGCN',
               'SpaGFT', 'GPcounts', 'Sepal', 'BOOST-GP']

dataset_list = ['151507', '151508', '151509', '151510', '151669', '151670', 
                '151671', '151672', '151673', '151674', '151675', '151676']

In [5]:
def get_genes(df, method, n_svgs):
    if method == 'nnSVG':
        df = df.sort_values(['LR_stat'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['gene_id']]
        
    elif method == 'Spanve':
        df = df.sort_values(['ent'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
    
    elif method == 'SPARK-X':
        df = df.sort_values(['adjustedPval'])
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
    
    elif method == 'SPARK':
        df = df.sort_values(['adjusted_pvalue'])
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'SpatialDE2':
        df = df.sort_values(['FSV'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
    
    elif method == 'SpatialDE':
        df = df.sort_values(['FSV'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'SOMDE':
        df = df.sort_values(['FSV'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'MoranI':
        df = df.sort_values(['I'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'scGCO':
        df = df.sort_values(['fdr'])
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'SpaGCN':
        df = df.sort_values(['pvals_adj'])
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]

    elif method == 'SpaGFT':
        df = df.sort_values(['gft_score'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]

    elif method == 'GPcounts':
        df['log_likelihood_ratio'] = df['log_likelihood_ratio'].fillna(0)
        df = df.sort_values(['log_likelihood_ratio'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
    
    elif method == 'Sepal':
        df = df.sort_values(['sepal_score'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
        
    elif method == 'BOOST-GP':
        df = df.sort_values(['PPI'], ascending=False)
        df = df.head(n_svgs).reset_index()
        df = df[['Unnamed: 0']]
    else:
        print(f'unknown method: {method}')
        
    return df

In [6]:
for n_svgs in [500, 1000, 1500, 2000]:
    for method in method_list:
        os.makedirs(f'{out_dir}/{method}', exist_ok=True)

        for dataset in dataset_list:
            input_filename = f'{input_dir}/{method}/{dataset}.csv'

            df = pd.read_csv(input_filename)
            df = get_genes(df, method, n_svgs=n_svgs)
            df.columns = ['gene']
            df.to_csv(f'{out_dir}/{method}/{dataset}_{n_svgs}.csv', header=True, index=False)

In [7]:
# get HVGs as control
os.makedirs(f'{out_dir}/HVG', exist_ok=True)

for n_genes in [500, 1000, 1500, 2000]:
    for dataset in dataset_list:
        adata = sc.read_h5ad(f"../../results/05_snakemake_clustering/data/{dataset}.h5ad")
        sc.pp.highly_variable_genes(adata, n_top_genes=n_genes, layer='counts', flavor='seurat_v3')
        df = adata.var[adata.var['highly_variable']]
        df = df.reset_index()
        df = df[['index']]
        df.columns = ['gene']
        df.to_csv(f'{out_dir}/HVG/{dataset}_{n_genes}.csv', header=True, index=False)